# Airfoil Model Analysis




This Jupyter Notebook is designed to analyze airfoil models using machine learning techniques. The notebook includes the following key steps:

1. **Importing Libraries and Setting Up Paths**: Essential libraries such as `pandas`, `numpy`, `torch`, and `matplotlib` are imported. Paths for project data, models, and results are defined.

2. **Device Configuration**: The notebook checks for GPU availability and sets the device accordingly.

3. **Utility Functions**: Functions for data preparation, downsampling, and organizing data are imported from external utility scripts.

4. **Data Preparation**: The `prep_data` function is defined to load and preprocess airfoil coordinate and polar data.

5. **Model Initialization**: Pre-trained models for predicting lift coefficient ($C_l$) and drag coefficient ($C_d$) are loaded and set to evaluation mode.

6. **Evaluation Metrics**: Functions to compute Mean Absolute Percentage Error (MAPE) and Relative L2 Norm Error are defined.

7. **$C_l$ Model Analysis**: The notebook evaluates the $C_l$ model on test data, computes evaluation metrics, and optionally plots the results.

8. **$C_d$ Model Analysis**: Similarly, the $C_d$ model is evaluated, and metrics are computed and plotted if required.

9. **Results Saving**: Evaluation results are saved to CSV files if the `SAVE_RESULTS` flag is set.

This notebook provides a comprehensive workflow for analyzing airfoil models, from data preparation to model evaluation and result visualization.


In [ ]:
import os
import pandas as pd
import numpy as np
import sys
import re

import torch
import matplotlib.pyplot as plt

In [ ]:
PLOT_RESULTS = False
SAVE_RESULTS = False
SAVE_PLOTS = False

project_path = '/mnt/e/eVTOL_model/eVTOL-VehicleModel/'
data_path = project_path + 'data/airfoil_data/'
model_path = project_path + 'trained_models/models/airfoil/'
save_path = project_path + 'result/airfoil_model/'

In [ ]:
# Check if GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
sys.path.append(project_path + '/src')


# Import necessary functions
from utility_functions import downsample_to_35
from utility_functions import organize_data

# Import all the models
from af_escnn_cl import ESCNN_Cl
from af_escnn_cd import ESCNN_Cd


In [ ]:
def prep_data(root, keyword=None):

    # Initialization of arrays
    # Coordinates
    x_i = [] # Initial coordinates - before downsizing
    y_i = []

    # Polars
    alphas = []
    Cls = []
    Cds = []
    Cms = []

    if keyword:
        # lists of files in each dir
        coord_files = [f for f in os.listdir(root) if f == (keyword+'_coordinates.dat')]
        polar_files = [f for f in os.listdir(root) if f == ('xf-'+keyword+'-il-1000000.csv')]
    else:
        # lists of files in each dir
        coord_files = [f for f in os.listdir(root) if f.endswith('_coordinates.dat')]
        polar_files = [f for f in os.listdir(root) if f.endswith('.csv')]

    # Extract base names from coordinate files
    coord_bases = {re.sub(r'\_coordinates.dat$', '', f) for f in coord_files}
    polar_bases = {}
    for polar_file in polar_files:
        match = re.match(r'xf-(.*)-il-1000000\.csv$', polar_file)
        if match:
            base_name = match.group(1)
            polar_bases[base_name] = polar_file
    # print(polar_bases)
    for base_name in coord_bases:
        if base_name in polar_bases:
            coord_file = f"{base_name}_coordinates.dat"
            polar_file = polar_bases[base_name]

            coordinate_data = np.loadtxt(root+coord_file)
            # polar_data = np.loadtxt(root+polar_file, skiprows=12)
            polar_data = pd.read_csv(root+polar_file, skiprows=10)
            polar_data = polar_data[(polar_data['Alpha'] >= -2) & (polar_data['Alpha'] <= 10)]
            # print(len(polar_data))

            # Coordinates
            x = []
            y = []

            # Polars
            alpha = polar_data['Alpha'].values
            Cl = polar_data['Cl'].values
            Cd = polar_data['Cd'].values
            Cm = polar_data['Cm'].values

            # print(alpha)

            for i in range(0, len(coordinate_data)):
                np.array(x.append(float(coordinate_data[i][0]))) 
                np.array(y.append(float(coordinate_data[i][1])))
                
            if len(x) >= 35:    # Only consider the files with more than 35 coordinates
                x_i.append(x)
                y_i.append(y)

                alphas.append(alpha)
                # Cls.append(Cl)
                # Cds.append(Cd)

                for num_val in range(len(Cl)):
                    Cls.append(Cl[num_val])
                    Cds.append(Cd[num_val])
                    Cms.append(Cm[num_val])

    return x_i, y_i, Cls, Cds, Cms, alphas

In [ ]:
# Initialize airfoil models
root_airfoilModelsTrained = model_path
# root_scalers = '/mnt/e/eVTOL_model/eVTOL-VehicleModel/trained_models/scalers/'

# Load the model weights
af_model_ESCNN_Cl = ESCNN_Cl()
af_model_ESCNN_Cl.load_state_dict(torch.load(root_airfoilModelsTrained + '2024-11-18_model_Cl_ESCNN_lr1e-05_e1500_rbf170_convL4.pth'))
af_model_ESCNN_Cl = af_model_ESCNN_Cl.to(device)
af_model_ESCNN_Cl.eval()

# Load the model weights
af_model_ESCNN_Cd = ESCNN_Cd()
af_model_ESCNN_Cd.load_state_dict(torch.load(root_airfoilModelsTrained + '2024-11-18_model_Cd_ESCNN_lr5e-05_e250_convL3.pth'))
af_model_ESCNN_Cd = af_model_ESCNN_Cd.to(device)
af_model_ESCNN_Cd.eval()


In [ ]:
def mape(y_true, y_pred):
    """Compute Mean Absolute Percentage Error (MAPE)"""
    mask = y_true != 0  # Avoid division by zero
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

def relative_l2_norm(y_true, y_pred):
    """Compute Relative L2 Norm Error (ε)"""
    mask = y_true != 0  # Avoid division by zero
    numerator = np.linalg.norm(y_pred[mask] - y_true[mask], ord=2)  # ||pred - true||_2
    denominator = np.linalg.norm(y_true[mask], ord=2)  # ||true||_2
    return (numerator / denominator) * 100

## $C_l$ model analysis

In [ ]:
from sklearn.metrics import r2_score

root_test = data_path + "/testing_database/"

coord_files = [f for f in os.listdir(root_test) if f.endswith('_coordinates.dat')]
coord_bases = {re.sub(r'\_coordinates.dat$', '', f) for f in coord_files}


# Initialize lists to store evaluation results
l2_norm_cl_list= []
r2_cl_list = []
airfoil_names = []

for keyword in coord_bases:
    x_t, y_t, Cls_t, Cms_t, Cds_t, alphas_t = prep_data(root_test, keyword)
    Cls_t = np.array(Cls_t, dtype=float)
    Cds_t = np.array(Cds_t, dtype=float)
    # Cms_t = np.array(Cms_t, dtype=float)
    
    x_f_t = [] # Final coordinates - after downsizing
    y_f_t = []
    for num_airfoil in range(0, len(x_t)):
        downsampled_x = downsample_to_35(x_t[num_airfoil])
        downsampled_y = downsample_to_35(y_t[num_airfoil])

        x_f_t.append(downsampled_x)
        y_f_t.append(downsampled_y)
        
    Elements_t = organize_data(x_f_t, y_f_t, alphas_t)
    # print("Elements: ",Elements_t.shape)
    # print("Cls: ",Cls_t.shape)

    if Elements_t.shape != (0,):
        input_data_test = Elements_t

        # Convert to PyTorch tensors
        input_test = torch.tensor(input_data_test, dtype=torch.float32)

        # Move data to GPU
        input_test = input_test.to(device)

        # Evaluate the model on test dataset
        with torch.no_grad():
            Cl_eval = af_model_ESCNN_Cl.forward(input_test)
            
        Cl_eval = Cl_eval.cpu().detach().squeeze(1).numpy()  # Convert tensor to numpy array

        # Calculate the evaluation metrics
        l2_norm_cl = relative_l2_norm(Cls_t, Cl_eval)
        r2_cl = r2_score(Cls_t, Cl_eval)

        l2_norm_cl_list.append(l2_norm_cl)
        r2_cl_list.append(r2_cl)
        airfoil_names.append(keyword)

        print(f"Airfoil: {keyword}, relative L2 Norm Cd: {l2_norm_cl:.2f}%, R^2 Cd: {r2_cl:.4f}")

        # plotting
        if PLOT_RESULTS:
            plt.figure(figsize=(10, 6), dpi=300)
            plt.plot(alphas_t[0], Cl_eval, '--', color='black', linewidth=1.5, label='ESCNN Model')
            plt.plot(alphas_t[0],Cls_t, color='red', linewidth=1.5, label='XFOIL')

            plt.legend(loc='upper center', fontsize=16, ncol=2, fancybox=True)
            plt.grid(True, linestyle='--', linewidth=0.5)
            plt.title(r'$C_l$ vs $\alpha$ for {} airfoil'.format(keyword), fontsize=20)
            plt.xlabel(r'AOA [$\alpha$]', fontsize=16)
            plt.ylabel(r'$C_l$', fontsize=16)

            if SAVE_PLOTS:
                # plt.gca().set_aspect('equal', adjustable='box')  # Maintain correct aspect ratio
                plt.savefig("{}cl_comparison_{}.pdf".format(save_path, keyword), format="pdf", dpi=300, bbox_inches="tight")

        


    else:
        continue

print("Mean l2_norm Cl: {:.2f}%".format(np.mean(l2_norm_cl_list)))
print("Mean R^2 Cl: {:.4f}".format(np.mean(r2_cl_list)))

mean_l2_norm = np.mean(l2_norm_cl_list)
mean_r2 = np.mean(r2_cl_list)

# Convert results to a DataFrame
results_df = pd.DataFrame({
    "Airfoil": airfoil_names,
    "Relative L2 Norm Cl (%)": l2_norm_cl_list,
    "R² Cl": r2_cl_list
})

# Append mean values to the DataFrame
mean_row = pd.DataFrame({
    "Airfoil": ["Mean"], 
    "Relative L2 Norm Cl (%)": [mean_l2_norm], 
    "R² Cl": [mean_r2]
})



if SAVE_RESULTS:
    results_df = pd.concat([results_df, mean_row], ignore_index=True)
    # Define CSV file path
    csv_filename = os.path.join(save_path, "Cl_evaluation_results.csv")

    # Save to CSV
    results_df.to_csv(csv_filename, index=False)

    print(f"Results (including mean values) saved to {csv_filename}")

## $C_d$ model analysis

In [ ]:
root_test = data_path + "/testing_database/"


coord_files = [f for f in os.listdir(root_test) if f.endswith('_coordinates.dat')]
coord_bases = {re.sub(r'\_coordinates.dat$', '', f) for f in coord_files}

# Initialize lists to store evaluation results
l2_norm_cd_list = []
r2_cd_list = []

for keyword in coord_bases:
    x_t, y_t, Cls_t, Cds_t, Cms_t, alphas_t = prep_data(root_test, keyword)
    Cls_t = np.array(Cls_t, dtype=float)
    Cds_t = np.array(Cds_t, dtype=float)
    Cms_t = np.array(Cms_t, dtype=float)
    
    x_f_t = [] # Final coordinates - after downsizing
    y_f_t = []
    for num_airfoil in range(0, len(x_t)):
        downsampled_x = downsample_to_35(x_t[num_airfoil])
        downsampled_y = downsample_to_35(y_t[num_airfoil])

        x_f_t.append(downsampled_x)
        y_f_t.append(downsampled_y)
        
    Elements_t = organize_data(x_f_t, y_f_t, alphas_t)
    # print("Elements: ",Elements_t.shape)
    # print("Cds: ",Cds_t.shape)

    if Elements_t.shape != (0,):
        input_data_test = Elements_t

        # Convert to PyTorch tensors
        input_data_test = torch.tensor(input_data_test, dtype=torch.float32)

        # Move data to GPU
        input_data_test = input_data_test.to(device)

        # Evaluate the model on test dataset
        with torch.no_grad():
            Cd_eval = af_model_ESCNN_Cd.forward(input_data_test)

        # Calculate the evaluation metrics
        Cd_eval = Cd_eval.cpu().numpy().flatten()

        l2_norm_cd = relative_l2_norm(Cds_t, Cd_eval)
        r2_cd = r2_score(Cds_t, Cd_eval)

        l2_norm_cd_list.append(l2_norm_cd)
        r2_cd_list.append(r2_cd)

        print(f"Airfoil: {keyword}, relative L2 Norm Cd: {l2_norm_cd:.2f}%, R^2 Cd: {r2_cd:.4f}")

        if PLOT_RESULTS:
            plt.figure(figsize=(10, 6), dpi=300)
            plt.plot(alphas_t[0], Cd_eval, '--', color='black', linewidth=1.5, label='ESCNN Model')
            plt.plot(alphas_t[0],Cds_t, color='red', linewidth=1.5, label='XFOIL')

            plt.legend(loc='upper center', fontsize=16, ncol=2, fancybox=True)
            plt.grid(True, linestyle='--', linewidth=0.5)
            plt.title(r'$C_d$ vs $\alpha$ for {} airfoil'.format(keyword), fontsize=20)
            plt.xlabel(r'AOA [$\alpha$]', fontsize=16)
            plt.ylabel(r'$C_d$', fontsize=16)

            if SAVE_PLOTS:
                # plt.gca().set_aspect('equal', adjustable='box')  # Maintain correct aspect ratio
                plt.savefig("{}cd_comparison_{}.pdf".format(save_path, keyword), format="pdf", dpi=300, bbox_inches="tight")

    else:
        continue

# Save the evaluation results
print("Mean l2_norm Cd: {:.2f}%".format(np.mean(l2_norm_cd_list)))
print("Mean R^2 Cd: {:.4f}".format(np.mean(r2_cd_list)))

mean_l2_norm = np.mean(l2_norm_cd_list)
mean_r2 = np.mean(r2_cd_list)

# Convert results to a DataFrame
results_df = pd.DataFrame({
    "Airfoil": airfoil_names,
    "Relative L2 Norm Cd (%)": l2_norm_cd_list,
    "R² Cd": r2_cd_list
})

# Append mean values to the DataFrame
mean_row = pd.DataFrame({
    "Airfoil": ["Mean"], 
    "Relative L2 Norm Cd (%)": [mean_l2_norm], 
    "R² Cd": [mean_r2]
})



if SAVE_RESULTS:
    results_df = pd.concat([results_df, mean_row], ignore_index=True)
    # Define CSV file path
    csv_filename = os.path.join(save_path, "Cd_evaluation_results.csv")

    # Save to CSV
    results_df.to_csv(csv_filename, index=False)

    print(f"Results (including mean values) saved to {csv_filename}")